In [ ]:
import os
import sys
import shutil
import nbformat
import pandas as pd
import subprocess
import tkinter as tk
from tkinter import ttk
from tqdm import tqdm
from IPython.display import display
from IPython.display import FileLink

In [ ]:
def decide():
    def get_values():
        global run_scripts_value, load_data_value
        run_scripts_value = run_scripts_var.get()
        load_data_value = load_data_var.get()
        dialog.destroy()

    dialog = tk.Tk()
    dialog.title("Configuration")

    global run_scripts_var, load_data_var
    run_scripts_var = tk.BooleanVar(value=False)
    load_data_var = tk.BooleanVar(value=False)


    run_scripts_var = tk.BooleanVar()
    load_data_var = tk.BooleanVar()


    run_scripts_checkbox = ttk.Checkbutton(dialog, text="Run Scripts", variable=run_scripts_var)
    load_data_checkbox = ttk.Checkbutton(dialog, text="Load Data", variable=load_data_var)


    confirm_button = ttk.Button(dialog, text="👍", command=get_values)


    run_scripts_checkbox.grid(row=0, column=0, padx=10, pady=5, sticky="w")
    load_data_checkbox.grid(row=1, column=0, padx=10, pady=5, sticky="w")
    confirm_button.grid(row=2, column=0, padx=10, pady=10)

    dialog.mainloop()

In [ ]:
def select_animals(df_data):
    exp_ids = []
    boolean_var = []

    def valider():
        for i, var in enumerate(var_etats):
            boolean_var.append(var.get())
            if var.get()==True:
                exp_ids.append(noms[i])
                print(f"{noms[i]} ({descriptions[i]})")
        root.destroy()

    def check_all():
        for var in var_etats:
            var.set(True)

    def uncheck_all():
        for var in var_etats:
            var.set(False)

    def update_checkboxes(description):
        for i, desc in enumerate(descriptions):
            if desc == description:
                var_etats[i].set(var_description_states[description].get())

    noms = df_data['exp_id']
    descriptions = df_data['description']

    root = tk.Tk()
    root.title("Interface avec cases à cocher")

    root.geometry("1000x500")

    canvas = tk.Canvas(root)
    scrollbar = ttk.Scrollbar(root, orient="vertical", command=canvas.yview)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(
            scrollregion=canvas.bbox("all")
        )
    )
    canvas.create_window((0, 0), window=scrollable_frame, anchor="center")
    canvas.configure(yscrollcommand=scrollbar.set)

    var_etats = []

    for i, nom in enumerate(noms):
        var = tk.BooleanVar()
        var_etats.append(var)

        checkbox = tk.Checkbutton(scrollable_frame, text=f"{nom}", font=("Arial", 11, "bold"), fg='blue', variable=var)
        checkbox.grid(row=i, column=0, sticky="w")
        label = tk.Label(scrollable_frame, text=f"({descriptions[i]})", font=("Arial", 10, "italic"))
        label.grid(row=i, column=1, sticky="w")

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    btn_valider = tk.Button(root, text="👍", command=valider, font=(20))
    btn_valider.pack(side="bottom", padx=5, pady=35)

    btn_check_all = tk.Button(root, text="✅ All", command=check_all)
    btn_check_all.pack(side="bottom", padx=5, pady=5)

    btn_uncheck_all = tk.Button(root, text="❌ All", command=uncheck_all)
    btn_uncheck_all.pack(side="bottom", padx=5, pady=5)

    var_description_states = {}
    for description in set(descriptions):
        var_description_states[description] = tk.BooleanVar()
        checkbox = tk.Checkbutton(root, text=description, variable=var_description_states[description], command=lambda desc=description: update_checkboxes(desc))
        checkbox.pack(side="top", padx=5, pady=5)

    root.mainloop()

    return exp_ids, boolean_var

In [ ]:
def write_on_notebook(notebook_path, text, cell_number):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    nb['cells'][cell_number]['source'] = text

    with open(notebook_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

In [1]:
# def multi_animal_analysis(df_data):
decide()

if run_scripts_value:
    print("RUNNING THE SCRIPTS...")
    exp_ids, boolean_var = select_animals(df_data)
    raw_data_paths = df_data.loc[boolean_var, 'raw_data_path']
    scripts = df_data.loc[boolean_var, 'script']
    scripts = [script.split(', ') for script in scripts]

    # %matplotlib qt
    for raw_data_path, exp_id, animal_script in tqdm(zip(raw_data_paths, exp_ids, scripts), total=len(exp_ids), desc='Processing the multi-animal analysis...', leave=True):
        print('\n')
        nom_fichier_sortie = fr'C:\Users\gbouvier\Documents\Analyses\{exp_id}\output.txt'
        if not os.path.exists(os.path.dirname(nom_fichier_sortie)):
            os.makedirs(os.path.dirname(nom_fichier_sortie), exist_ok=True)
        sys.stdout = open(nom_fichier_sortie, 'w')
        for script in animal_script:
            os.makedirs(fr'C:\Users\gbouvier\Documents\Analyses\{exp_id}\scripts', exist_ok=True)
            filename = os.path.basename(script)
            new_script = fr"C:\Users\gbouvier\Documents\Analyses\{exp_id}\scripts\{filename.replace('.ipynb', f'_{exp_id}.ipynb')}"
            shutil.copyfile(script, new_script)

            write_on_notebook(new_script, f'raw_data_path = r"{raw_data_path}"', 2)

            %run {new_script}
            # subprocess.run(['jupyter', 'nbconvert', '--to', 'html', new_script])
            
            plt.close('all')
            
            print('-'*100)
            print('\n')
        
        foo = saving_folder
        while exp_id not in os.path.basename(foo):
            foo = os.path.dirname(foo)
        if os.name == 'posix':  # Unix (Linux, macOS)
            subprocess.Popen(['xdg-open', foo])
        elif os.name == 'nt':  # Windows
            subprocess.Popen(['explorer', foo])
        sys.stdout = sys.__stdout__
        display(FileLink(nom_fichier_sortie))
        print('\n')
    # %matplotlib inline

if load_data_value:
    global rotation_data, phototagging_data
    rotation_data = {}
    phototagging_data = {}

    print("LOADING THE DATA...")
    exp_ids, boolean_var = select_animals(df_data)
    analysis_data_paths = df_data.loc[boolean_var, 'analysis_data_path']
    
    for exp_id, analysis_data_path in tqdm(zip(exp_ids, analysis_data_paths), total=len(exp_ids), desc='Loop over animals'):
        for file in os.listdir(analysis_data_path):
            if file.endswith('.npz') and 'rotation' in file:
                rotation_data[exp_id] = np.load(os.path.join(analysis_data_path, file), allow_pickle=True)

                rotation_data[exp_id] = {key: rotation_data[exp_id][key].copy() for key in rotation_data[exp_id].keys()}

                for item in rotation_data[exp_id]:
                    if rotation_data[exp_id][item].size == 1:
                        rotation_data[exp_id][item] = rotation_data[exp_id][item].item()
            elif file.endswith('.npz') and 'phototagging' in file:
                phototagging_data[exp_id] = np.load(os.path.join(analysis_data_path, file), allow_pickle=True)
                phototagging_data[exp_id] = {key: phototagging_data[exp_id][key].copy() for key in phototagging_data[exp_id].keys()}
                for item in phototagging_data[exp_id]:
                    if phototagging_data[exp_id][item].size == 1:
                        phototagging_data[exp_id][item] = phototagging_data[exp_id][item].item()
            


            # if file.endswith('.pickle') and 'rotation' in file:
            #     rotation_data[exp_id] = pd.read_pickle(os.path.join(analysis_data_path, file))
            # elif file.endswith('.pickle') and 'phototagging' in file:
            #     phototagging_data[exp_id] = pd.read_pickle(os.path.join(analysis_data_path, file))

    # data = load_data(data)
    # return data

#run_scripts_value, load_data_value = False, False

IndentationError: unexpected indent (1760160310.py, line 64)